# Packages

In [19]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd 
import numpy as np
import selenium
from tkinter import *
import tkinter as tk
from tkinter import ttk
from selenium.webdriver.chrome.options import Options# for suppressing the browser
import time


# Partie Marmiton

In [20]:
def Interface_Saisie():
    # Generer une interface pour y entrer le nom du plat que l'on souhaite faire
    fen = Tk() 
    image = PhotoImage(file='Image.png', master=fen) 
    # Importation de l'image
    can = Canvas(fen, width=500, height=400, bg='pink')
    can.grid(column=0, row=0, columnspan=3, padx=10, pady=10)
    can.create_image((150, 200), image=image)
    l=[]
    def quit():
        """ Récupérer ce qui a été saisi par l'utilisateur"""
        plat=nom.get()
        l.append(plat)
        fen.destroy()
#################### CREATION DE L'INTERFACE ####################
    libelle = Label(can, text='Alors que vous voulez vous préparer aujourdhui:')
    libelle.pack()
    nom = Entry(can)
    nom.pack(pady=180, padx=180)
    b1 = Button(fen, text='Valider votre choix', command=quit)
    b1.grid(column=0, row=1, padx=200, pady=10)
    fen.mainloop()
    return(l[0])

In [21]:
def saisir_recette(plat): 
    # Permet de naviguer vers la recette choisie sur le site de marmiton.
    driver = webdriver.Chrome()
    # commenter cette commande si on souhaite cacher le driver lors de l'execution
    driver.set_window_position(-10000, 0)
    url = 'https://www.marmiton.org/recettes/recherche.aspx?aqt='+plat
    driver.get(url)
    driver.find_element_by_id('didomi-notice-agree-button').click()
    html = driver.page_source
    bs = BeautifulSoup(html, "html.parser")
    driver.close()
    return bs

In [22]:
def extract(bs): 
    # cette fonction permet d'extraire l'ensemble des infos ci-dessous liées au plat choisit 
    # (!!! retourne seulement les 3 premières recettes)
    nom = list() # le nom du plat
    lien = list() # le lien vers la recette 
    note = list() # la note de la recette 
    review = list() # le nbr de personnes qui ont mis une remarque 
    nom = list(map(lambda i: i.get_text(), bs.find_all('h4', {'class':'MRTN__sc-30rwkm-0 dJvfhM'})))[:3]
    lien = list(map(lambda i: "https://www.marmiton.org"
                    + i.attrs['href'], bs.find_all('a', {'class':'MRTN__sc-1gofnyi-2 gACiYG'})))[:3]
    note = list(map(lambda i: i.get_text(), 
                    bs.find_all('span', {'class':'SHRD__sc-10plygc-0 jHwZwD'})))[:3]
    review = list(map(lambda i:i.get_text(), 
                      bs.find_all('div', {'class':'MRTN__sc-30rwkm-3 fyhZvB'})))[:3]
    df = pd.DataFrame(np.column_stack([nom, lien, note, review]),
                      columns=['nom', 'lien', 'note', 'review'])
    return df # Les differentes informations sont mise dans une dataframe

In [23]:
def extract_inf(df, bs): # Cette fonction retourne le temps, la difficulté, le cout d'une recette 
    time = list()
    diff = list()
    cout = list()
    for i in range(0, len(df)):
        url = df['lien'][i]
        html=urlopen(url)
        bs = BeautifulSoup(html, "html.parser")
        time.append(bs.find_all('p', {'class':'RCP__sc-1qnswg8-1 iDYkZP'})[0].get_text())
        diff.append(bs.find_all('p', {'class':'RCP__sc-1qnswg8-1 iDYkZP'})[1].get_text())
        cout.append(bs.find_all('p', {'class':'RCP__sc-1qnswg8-1 iDYkZP'})[2].get_text())
    df = pd.DataFrame(np.column_stack([time, diff, cout]), 
                      columns=['Temps de préparation', 'difficulté', 'cout'])
    return df # Le resultat retourné est sous forme de dataframe

In [24]:
def tableau_Affichage(Liste_plats): 
    # cette fonction affiche une interface qui résume les recettes
    # et demande à l'utilisateur d'en choisir une
    fenetre = Tk()
    fenetre.title('Affichage des plats')
    style = ttk.Style(fenetre)
    # Création et remplissage du tableau
    libelle = Label(fenetre, text='PLATS')
    libelle.pack(padx=10, pady=10)
    tableau = ttk.Treeview(fenetre, 
                           columns=('Num', 'Nom', 'Note', 'Temps', 'difficulté', 'Cout'))
    tableau.heading('Num', text='Num')
    tableau.heading('Nom', text='Nom')
    tableau.heading('Note', text='Note')
    tableau.heading('Temps', text='Temps')
    tableau.heading('difficulté', text='difficulté')
    tableau.heading('Cout', text='Cout')               
    tableau['show'] = 'headings' 
    tableau.pack(padx=10, pady=(0, 10))
    for i in range(0, len(Liste_plats)):
        tableau.insert('', 'end', iid=i, values=(i, Liste_plats[i][0], 
                                                    Liste_plats[i][2], 
                                                    Liste_plats[i][4], 
                                                    Liste_plats[i][5],
                                                    Liste_plats[i][6]))
    l = []
    def OK():
        # Récupérer ce qui a été saisi par l'utilisateur
        choice = nom.get()
        l.append(choice)
        fenetre.destroy()
    
    libelle = Label(fenetre, text='Choisissez un plat :')
    libelle.pack()
    nom = Entry(fenetre)
    nom.pack()
    btn = Button(fenetre, text="Ok", command=OK)
    btn.pack()
    fenetre.mainloop()
    return (l[0])
    

In [25]:
def extract_ing(df_f, i):
    # Extraire les ingredients de la recette choisie
    dic = {}
    url = df_f['lien'][i]
    html = urlopen(url)
    bs = BeautifulSoup(html, "html.parser")
    poid = bs.find_all('div', {'class':'RCP__sc-8cqrvd-0 hSorOY'})
    ing = bs.find_all('span', {'class':'SHRD__sc-10plygc-0 kWuxfa'})
    for i in range(0, len(poid)):
        ingredient = ing[i].get_text()
        ingredient = ingredient.replace("de", "", 1)
        ingredient = ingredient.replace("d'", "", 1)
        dic[poid[i].get_text().replace('\xa0', '').replace('\xa0', '')] = ingredient
        # parfois get_text() retourne '\xa0' and '\xa0g'
    return dic

In [26]:
def extract_rec (df_f, i): 
    # extraire la recette
    url = df_f['lien'][i]
    driver.get(url)
    html = driver.page_source
    bs = BeautifulSoup(html, "html.parser")
    nbr_steps = len(bs.find_all('h3', {'class':'RCP__sc-1wtzf9a-1 ikYBNp'}))
    for i in range(0, nbr_steps):
        print(bs.find_all('h3', {'class':'RCP__sc-1wtzf9a-1 ikYBNp'})[i].get_text())
        print(bs.find_all('p', {'class':'RCP__sc-1wtzf9a-3 jFIVDw'})[i].get_text())
    
    

# Partie Monoprix

In [27]:
def login_monoprix(email, mdp): 
    # Permet de se connecter à monoprix
    option = webdriver.ChromeOptions()
    # Removes navigator.webdriver flag
    option.add_argument('--disable-blink-features=AutomationControlled')
    # Open Browser
    driver = webdriver.Chrome(executable_path='chromedriver.exe',options=option)
    driver.get('https://www.monoprix.fr/login')
    try:
        # log in:
        time.sleep(5)
        driver.find_element_by_id('onetrust-close-btn-container').click() # fermer la fenetre termes de navigation
        driver.find_element_by_name('email').clear()
        driver.find_element_by_name('email').send_keys(email)
        driver.find_element_by_name('password').clear()
        driver.find_element_by_name('password').send_keys(mdp)
        driver.implicitly_wait(10)
        driver.find_element_by_tag_name('button').click()
        return driver
    except:
        # log in:
        time.sleep(5)
        driver.find_element_by_id('onetrust-close-btn-container').click() # fermer la fenetre termes de navigation
        driver.find_element_by_name('email').clear()
        driver.find_element_by_name('email').send_keys(email)
        driver.find_element_by_name('password').clear()
        driver.find_element_by_name('password').send_keys(mdp)
        driver.implicitly_wait(10)
        driver.find_element_by_tag_name('button').click()
        return driver

In [28]:
def purchase_item(produit): 
    #permet d'acheter le premier article
    try:
        #produit represente l'ingredient que l'on souhaite acheter
        driver.get('https://www.monoprix.fr/courses-en-ligne')
        driver.find_element_by_tag_name('input').send_keys(produit) # rajoute x à la barre de recherche 
        driver.implicitly_wait(5)
        driver.find_element_by_class_name('total-result').click() # valide la recherche 
        driver.find_elements_by_class_name('add-to-cart-button__add-button')[0].click()
        time.sleep(3)
        test = driver.find_elements_by_class_name('content')
        time.sleep(3)
        try:
            test[14].find_elements_by_tag_name('button')[0].click() 
        except:
            test[14].find_elements_by_tag_name('button')[0].click()
    except:
        driver.get('https://www.monoprix.fr/courses-en-ligne')
        driver.find_element_by_tag_name('input').send_keys(produit)
        driver.implicitly_wait(5)
        driver.find_element_by_class_name('total-result').click()
        driver.find_elements_by_class_name('add-to-cart-button__add-button')[0].click()
        driver.implicitly_wait(10)
        test = driver.find_elements_by_class_name('content')
        driver.implicitly_wait(10)
        test[14].find_elements_by_tag_name('button')[0].click() 

In [29]:
def collect_date_time(): 
    # permet de collecter les differents date de livraison possible
    date = driver.find_element_by_class_name('days-label').find_elements_by_tag_name('div')
    time = driver.find_element_by_class_name('table-content').find_elements_by_tag_name('table')
    horaires = list()
    for i in date: 
        for j in time:
           horaires.append([k.text 
                            for  k in j.find_element_by_tag_name('tbody').find_elements_by_tag_name('td') 
                            if 'positive' in k.get_attribute('class')])
    horaires_Date_1 = horaires[0]+horaires[1]+horaires[2] 
    horaires_Date_2 = horaires[3]+horaires[4]+horaires[5]
    horaires_Date_3 = horaires[6]+horaires[7]+horaires[8]
    # horaires_Date_4 = horaires[9]+horaires[10]+horaires[11]
    pickable_dates = {}
    pickable_dates[date[0].text.replace('\n', ' ')] = horaires_Date_1
    pickable_dates[date[1].text.replace('\n', ' ')] = horaires_Date_2
    pickable_dates[date[2].text.replace('\n', ' ')] = horaires_Date_3
    # pickable_dates[date[3].text.replace('\n', ' ')] = horaires_Date_4
    return pickable_dates

#{'Lundi 14/02': ['dès 09:00', 'dès 10:00', 'dès 11:00', ..., 'dès 18:00', 'dès 19:00'], 'Mardi 15/02':[....]

In [30]:
# Partie tkinter : Horaires
def Afficher_Horaires(L_heure): 
    # Generer une interface qui permet à l'utilisatuer de sélectionner la date et l'horaire qui lui convient
    window = Tk()
    window.title('Combobox') 
    window.geometry('300x200')   
    Time_choose = []
    l1 = Label(window, text="Choisit une heure")
    l1.grid(column=0, row=0)
    # Une liste qui est censé compporter les horaires possibles
    Horaire = []
    for cle,valeur in L_heure.items():
        # Récupération des dates et des heures pour les mettre dans une liste
        for i in range (0, len(valeur)):
            Horaire.append(cle + " " + valeur[i])
           
    def OK():
        l2 = Label(window, text="")
        l2.grid(column=0, row=8)
        Jour_select = (cmb.get()[0:-10])
        Time_choose.append(cmb.get()[0:-10])
        Time_choose.append(cmb.get()[-10:-1] + cmb.get()[-1])
        Heure_select = (cmb.get()[-11:-1] + cmb.get()[-1])
        l2.configure(text=cmb.get())
        window.destroy()
        
    cmb = ttk.Combobox(window, values=Horaire, width=30)
    cmb.grid(column=0, row=1)
    cmb.current(0)
    btn = Button(window, text="OK", command=OK)
    btn.grid(column=2, row=1)
    window.mainloop()
    return (Time_choose)

In [31]:
def extract_nbr(x): 
    # permet de supprimer cetains caractéres et mots redondant et non pertinent dans l'horaire
    x = x.replace('dès', '')
    x = x.replace('0', '')
    x = x.replace(':', '')
    x = x.replace(' ', '')
    return int(x)

In [32]:
def choix_creneau(Jour_select, heure_select): 
    # valide le créneau qui a été choisi par l'utilisaeur sur le site de monoprix
    heure_select = heure_select[1:]
    d = [i.text.replace('\n', ' ') 
         for i in driver.find_element_by_class_name('days-label').find_elements_by_tag_name('div')]
   
    driver.find_element_by_class_name('days-label').find_elements_by_tag_name('div')[d.index(Jour_select)].click()
    if (extract_nbr(heure_select)<=11):
        index = [i.text for i in driver.find_elements_by_class_name('hours-group__content__body')[0].find_elements_by_tag_name('td')].index(heure_select)
        driver.find_elements_by_class_name('hours-group__content__body')[0].find_elements_by_tag_name('td')[index].click()
    elif (extract_nbr(heure_select)>=12 and extract_nbr(heure_select)<=16):
        index = [i.text for i in driver.find_elements_by_class_name('hours-group__content__body')[1].find_elements_by_tag_name('td')].index(heure_select)
        driver.find_elements_by_class_name('hours-group__content__body')[1].find_elements_by_tag_name('td')[index].click()
    else:
        index = [i.text for i in driver.find_elements_by_class_name('hours-group__content__body')[2].find_elements_by_tag_name('td')].index(heure_select)
        driver.find_elements_by_class_name('hours-group__content__body')[2].find_elements_by_tag_name('td')[index].click()
    

In [33]:
def check_item(produit): 
    # vérifie si un produit est disponible sur le site de monoprix  
    time.sleep(2)
    try:
        driver.find_element_by_tag_name('input').clear()
    except:
        pass
    driver.find_element_by_tag_name('input').send_keys(produit)
    time.sleep(2)
    try:
        voir_boutton = driver.find_element_by_class_name('total-result')
        return True
    except: 
        return False

In [34]:
def add_to_basket(produit): 
    # Ajouter les articles dans le panier. 
    # RQ: cette fonction permet d'ajouter tous les articles dans le panier
    # sauf le premier article puisque il requiert de choisir un horaire.
    driver.find_element_by_tag_name('input').clear()
    time.sleep(3)
    driver.find_element_by_tag_name('input').send_keys(produit)
    time.sleep(3)
    try:
        driver.find_element_by_class_name('total-result').click()
        time.sleep(3)
        driver.find_element_by_class_name('grocery-item__price-cart').find_element_by_tag_name('img').click()
    except:
        pass

In [35]:
def tableau_fin(not_found_prod): 
    # Afficher les produits qui ne sont pas disponibles
    fin = Tk()
    fin.title('Affichage des plats')
    style = ttk.Style(fin)

    libelle = Label(fin, text='INGREDIENTS NON DISPONIBLES')
    libelle.pack(padx=10, pady=10)
    tableau = ttk.Treeview(fin, columns=('Ingredients'))
    tableau.heading('Ingredients', text='Ingredients')
    tableau['show'] = 'headings'
    tableau.pack(padx=10, pady=(0, 10))
    for i in range(0, len(not_found_prod)):
        tableau.insert('', 'end', iid=i, values=(not_found_prod[i]))
        
    def OK():
        # Récupérer ce qui a été saisi par l'utilisateur
        fin.destroy()
    btn = Button(fin, text="Ok", command=OK)
    libelle = Label(fin, text="Les ingrédients suivants ne sont pas disponible, vérifer ensuite votre panier")
    libelle.pack()
    btn.pack()
    fin.mainloop()

In [36]:

Plat = Interface_Saisie()
bs = saisir_recette(Plat)
df = extract(bs)
df2 = extract_inf(df, bs)
df_f = pd.concat([df, df2], axis=1)
rows = [list(df_f.iloc[i]) for i in range(0,3)]

rec_choix = tableau_Affichage(rows)

ingr = extract_ing(df_f, int(rec_choix))
L_ingr = list(set([i for i in ingr.values()]))
# monoprix
time.sleep(3)
driver = login_monoprix("Ziedkebir9@gmail.com", "123456@@Zied")

#driver.set_window_position(-10000,0) 
# décommenter la commande pour cacher le driver lors de l'éxecution
""" Pour éviter une erreur potentiel le programme commence par faire
en sorte que le premier article est disponible sur monoprix 
Cela ce fait en échangeant la position du premier article 
avec un article qui est disponible sur le site """

time.sleep(3)
driver.get('https://www.monoprix.fr/courses-en-ligne') 
time.sleep(3)

for i in range(1, len(L_ingr)):
    if(check_item(L_ingr[0])!=True):
        rep = L_ingr[0]
        L_ingr[0] = L_ingr[i]
        L_ingr[i] = rep
        break
    else:
        break
        
driver.get('https://www.monoprix.fr/courses-en-ligne') 
purchase_item(L_ingr[0])
time.sleep(5)
x = Afficher_Horaires(collect_date_time())
choix_creneau(x[0], x[1])
# va contenir les produits qui n'ont pas été trouvé sur le site
not_found_prod = list() 

for i in range(1,len(L_ingr)):
    add_to_basket(L_ingr[i])
    

time.sleep(3)
not_found_prod = [i for i in L_ingr if check_item(i)==False]

driver.get('https://www.monoprix.fr/courses/tunnel/panier')    

tableau_fin(not_found_prod)

<ipython-input-21-77aa2c55416d>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('didomi-notice-agree-button').click()
<ipython-input-27-22ad5cf82aa8>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='chromedriver.exe',options=option)
<ipython-input-27-22ad5cf82aa8>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('onetrust-close-btn-container').click() # fermer la fenetre termes de navigation
<ipython-input-27-22ad5cf82aa8>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name('email').clear()
<ipython-input-27-22ad5cf82aa8>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name('email').send_

WebDriverException: Message: chrome not reachable
  (Session info: chrome=91.0.4472.77)
Stacktrace:
Backtrace:
	Ordinal0 [0x01003733+2504499]
	Ordinal0 [0x00F9C401+2081793]
	Ordinal0 [0x00EA24F0+1058032]
	Ordinal0 [0x00E97584+1013124]
	Ordinal0 [0x00E97D76+1015158]
	Ordinal0 [0x00E994D5+1021141]
	Ordinal0 [0x00E935F6+996854]
	Ordinal0 [0x00EA3830+1062960]
	Ordinal0 [0x00EF43D2+1393618]
	Ordinal0 [0x00EE5D4B+1334603]
	Ordinal0 [0x00EC22B4+1188532]
	Ordinal0 [0x00EC3149+1192265]
	GetHandleVerifier [0x0117FB8C+1512252]
	GetHandleVerifier [0x0122B0DF+2214031]
	GetHandleVerifier [0x01084BC3+484211]
	GetHandleVerifier [0x01083E69+480793]
	Ordinal0 [0x00FA218D+2105741]
	Ordinal0 [0x00FA66E8+2123496]
	Ordinal0 [0x00FA6827+2123815]
	Ordinal0 [0x00FAFB73+2161523]
	BaseThreadInitThunk [0x762EFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77D27A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77D27A6E+238]


In [37]:
driver = login_monoprix("Ziedkebir9@gmail.com", "123456@@Zied")

<ipython-input-27-22ad5cf82aa8>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='chromedriver.exe',options=option)
<ipython-input-27-22ad5cf82aa8>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('onetrust-close-btn-container').click() # fermer la fenetre termes de navigation
<ipython-input-27-22ad5cf82aa8>:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('onetrust-close-btn-container').click() # fermer la fenetre termes de navigation
<ipython-input-27-22ad5cf82aa8>:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name('email').clear()
<ipython-input-27-22ad5cf82aa8>:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
